# Загрузка Pandas и очистка данных

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [35]:
data = pd.read_csv('main_task.xls')
df = data.copy()
display(data.head())
display(data.info())

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 2.0+ MB


None

In [3]:
# Приведем разброс цен к числовому значению, где 1 - самый низкий, 
# а 3 - самый высокий

def price(value):
    if value == '$':
        return 1
    elif value == '$$ - $$$':
        return 2
    elif value == '$$$$':
        return 3
    else: 
        return value

price_range = df['Price Range'].apply(price)

# Заменим здесь пропуски на моду
price_range = price_range.fillna(price_range.mode()[0])



In [4]:
# Добавим информацию о разнообразии меню (количесвте кухонь)

cuisine_count = list()
cuisine_set = set()
for cuisine_obj in df['Cuisine Style']:
    if type(cuisine_obj) != float:
        cuisine_list = cuisine_obj[2:-2].split("', '")
        cuisine_count.append(len(cuisine_list))
        for cuisine in cuisine_list:
            cuisine_set.add(cuisine)
    else:
        cuisine_count.append(1)
        
cuisines = pd.Series(cuisine_count, name = 'Cuisine Counter')


In [5]:
# Создадим Dummy-переменные для разных кухонь
        
df['Cuisine Style'] = df['Cuisine Style'].dropna().apply(lambda x: x[2:-2].split("', '"))
cuisine_frame = pd.DataFrame(0, index = range(40000), columns = cuisine_set)
i = -1
for line in df['Cuisine Style']:
    i += 1
    if type(line) != float:
        for cuisine in line:
            cuisine_frame[cuisine][i] = 1
    else: 
        continue

In [6]:
# Создадим Dummy-переменные для городов

cities = pd.get_dummies(df[['City']])

In [36]:
# Создадим новый признак о разнице во времени между двумя последними отзывами

review_delta = list()
for review_obj in df['Reviews']:
    if review_obj == '[[], []]':
         review_delta.append(None)
    else:
        review = review_obj[2:-2].split("], [")
        review_list = [review[i][1:-1].split("', '") for i in range(2)]
        date_list = [datetime.strptime(review_list[1][i], '%m/%d/%Y') 
                     for i in range(len(review_list[1]))]
        review_delta.append(np.max(date_list) - np.min(date_list))

review_delta = pd.Series(review_delta).apply(lambda x: x.days)

In [8]:
# Избавимся от выбросов в timedelta отзывов

def way_too_big_delta(delta):
    IQR = review_delta.quantile(0.75) - review_delta.quantile(0.25)
    upper_limit = review_delta.mean() + 1.5*IQR
    lower_limit = review_delta.mean() - 1.5*IQR
    if (delta > upper_limit) or (delta < lower_limit):
        return None
    else:
        return delta
    
review_delta = review_delta.apply(way_too_big_delta)
review_delta = review_delta.fillna(review_delta.mean())

In [26]:
restaurant_id = df['Restaurant_id'].apply(lambda x: x[3:])

In [9]:
# Избавимся от колонок типа object
columns_to_drop = []

for column in data.columns:
    if data[column].dtype == 'O':
        columns_to_drop.append(column)

data = data.drop(columns_to_drop, axis = 1)

# Избавимся от пропусков
data = data.fillna(0)

data

,Ranking,Rating,Number of Reviews
0,5570.0,3.5,194.0
1,1537.0,4.0,10.0
2,353.0,4.5,688.0
3,3458.0,5.0,3.0
4,621.0,4.0,84.0
...,...,...,...
39995,500.0,4.5,79.0
39996,6341.0,3.5,542.0
39997,1652.0,4.5,4.0
39998,641.0,4.0,70.0


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [27]:
# Х - данные с информацией о ресторанах, 
# y - целевая переменная (рейтинги ресторанов)
X = pd.concat([data.drop(['Rating'], axis = 1), price_range, cuisines, 
               cuisine_frame, cities, review_delta, restaurant_id], axis = 1)
y = df['Rating']

In [28]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [29]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [30]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [31]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [32]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.20760050000000002
